In [2]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np

import geopandas as gpd
import seaborn as sns

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/google/auth/_default.py:68: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
import shapely
from shapely.geometry import LineString, MultiPoint
from shapely.ops import split, substring

In [4]:
import zlib

### get total peak headways!!

* since specified as "and", get both periods and take the min...

In [5]:
##check this...
am_peak = (6, 9)
pm_peak = (3, 7)

In [6]:
import datetime as dt

In [7]:
am_commute_hours = list(range(6,9))
pm_commute_hours = list(range(15,19))
commute_hours = am_commute_hours + pm_commute_hours

### Single Operator Shakedown

#### Flow

* if next shape contains existing segment, ignore and clip
* calculate new segments on non-overlap (which may be discontiguous?)

In [229]:
def split_every_km(geometry):
    lines = []
    ix = -1
    
    geometry = geometry.iloc[0]
    if hasattr(geometry, 'geoms'): ##check if MultiLineString
        linestrings = geometry.geoms
    else:
        linestrings = [geometry]
    for linestring in linestrings:
        ix += 1
        _first = True
        for i in range(0, int(linestring.length), 1000):
    #         print(i)
            if not _first:
                ix += 1
            lines.append((substring(linestring, i, i+1000), str(ix)))
    return lines

In [230]:
##TODO diagram scope, decide on explicit args vs. relying on subroutine scoping...

In [231]:
def find_stop_with_most_routes(segment, stops, route_count_by_stop):
    stops_in_seg = gpd.clip(stops, segment.geometry) 
#     print(f'stops_in_seg {stops_in_seg}')
    max_routes_stop = (stops_in_seg
                   >> inner_join(_, route_count_by_stop, on = "stop_id")
                   >> filter(_.n_routes == _.n_routes.max())
                  ).iloc[[0]]
#     display(max_routes_stop[['stop_id', 'n_routes']])
#     print(f'max routes stop: {max_routes_stop}')
    segment['stop_id'] = max_routes_stop['stop_id'].iloc[0]
    segment['n_routes'] = max_routes_stop['n_routes'].iloc[0]
    return segment

In [232]:
def get_operator_views(itp_id):
        shapes = tbl.gtfs_schedule.shapes() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        shapes = gpd.GeoDataFrame(shapes, 
                                  geometry = gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat),
                                  crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
        trips = tbl.gtfs_schedule.trips() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        stop_times = tbl.gtfs_schedule.stop_times() >> filter(_.calitp_itp_id == int(itp_id)) >> collect()
        stops = (tbl.gtfs_schedule.stops() 
                 >> filter(_.calitp_itp_id == itp_id)
                 >> select(_.stop_id, _.stop_lat, _.stop_lon)
                 >> collect())
        
        return shapes, trips, stop_times, stops

In [233]:
bbb_views = get_operator_views(300)

In [323]:
def single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop, shape_id, hqta):
    global _debug
    global _debug2, _debug3
    
    print(shape_id)
    
    single_shape = (shapes
         >> filter(_.shape_id == shape_id)
         >> mutate(shape_pt_sequence = _.shape_pt_sequence.astype('int64'))
         >> arrange(_.shape_pt_sequence) ##arrange, then convert to line to preserve order...
        )
    
    route_line = LineString(list(single_shape['geometry']))
    single_line = single_shape[['calitp_itp_id', 'shape_id', 'calitp_extracted_at']].iloc[[0]] ##preserve info cols
    single_line['geometry'] = route_line
    single_line = gpd.GeoDataFrame(single_line, crs='EPSG:6414')
        ## test a shapes df split into segments
    _debug = single_line
    if hqta.size != 0:
#         display(hqta.head(2))
#         display(single_line.head(2)
        already_calculated = hqta.dissolve(by='calitp_itp_id')
        single_line = single_line.overlay(already_calculated, how='difference') ##doesn't like this?
        if single_line.size == 0:
            print('no line')
            segments_with_max_stop = None
            return
#     else:
#         print('else!')
    segmented = pd.DataFrame()
    _debug2 = single_line
    for segment in split_every_km(single_line.geometry): ##working
        to_append = single_line.drop(columns=['geometry'])
        to_append['geometry'] = segment[0]
        to_append['segment_sequence'] = segment[1]
    #     print(to_append)
        segmented = segmented.append(to_append)

    segmented = segmented.astype({'calitp_itp_id': str})
    ## compute (hopefully unique) hash of segment id that can be used across routes/operators
    segmented['hqta_segment_id'] = segmented.apply(lambda x:
                        zlib.crc32((x.calitp_itp_id + x.shape_id + x.segment_sequence).encode('utf-8')),
                                                   axis=1)
    segmented.geometry = segmented.buffer(50) ##generous buffer for street/sidewalk width?
    
    try:
        _debug = segmented
        segments_with_max_stop = segmented.apply(find_stop_with_most_routes, axis=1,
                                                 args = (stops, route_count_by_stop))
    except IndexError:
        print('no stops')
        segments_with_max_stop = None
        return ##no stops in segment

    max_stop_times = (stop_times 
                 >> select(_.stop_id, _.trip_id, _.departure_time)
                 >> inner_join(_, segments_with_max_stop, on = "stop_id")
                )

        ##only works for whole hours, but should be fine?
    _debug3 = max_stop_times
    
    ##TODO debug/simplify
    max_stop_times['departure_hour'] = max_stop_times['departure_time'].apply(lambda x:
                                                                    dt.datetime.strptime(x, '%H:%M:%S').hour)

    max_stop_times = max_stop_times[max_stop_times['departure_hour'].isin(commute_hours)]
    max_stop_times['am_peak'] = max_stop_times['departure_hour'].apply(lambda x: x in am_commute_hours)
    max_stop_times['pm_peak'] = ~max_stop_times['am_peak']

    segment_peak_service = max_stop_times.groupby(['hqta_segment_id'])[['am_peak', 'pm_peak']].sum()
    segment_peak_service['am_peak'] = (segment_peak_service['am_peak'] / len(am_commute_hours)).round(1)
    segment_peak_service['pm_peak'] = (segment_peak_service['pm_peak'] / len(pm_commute_hours)).round(1)

    segment_peak_service['hq_transit_corr'] = segment_peak_service.apply(lambda x:
                              True if x.am_peak > 4 and x.pm_peak > 4 else False, axis=1)

    segment_peak_service.reset_index() ## All segments hq_corridors, which makes sense for BBB1

    single_hqta = segments_with_max_stop >> inner_join(_, segment_peak_service, on = 'hqta_segment_id')
    single_hqta.drop(columns=['calitp_extracted_at'], inplace=True)

    print(f'appending {single_hqta.shape}')
#     display(single_hqta)
    return single_hqta

In [324]:
def single_operator_hqta(views, hqta_df=None):
    global _result
    
    shapes, trips, stop_times, stops = views
    
    stops = gpd.GeoDataFrame(stops,
                             geometry = gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                             crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    distinct_routes = (trips
                   >> distinct(_.route_id, _.shape_id, _.direction_id, _keep_all = True)
                   >> select(_.calitp_itp_id, _.route_id, _.shape_id, _.direction_id, _.trip_id))
    route_count_by_stop = (stop_times
                         >> select(_.stop_id, _.trip_id)
                         >> inner_join(_, distinct_routes, on = "trip_id")
                         >> count(_.stop_id)
                         >> rename(n_routes = _.n)
                         >> arrange(-_.n_routes))

    ## for shape do single shape hqta...
#     all_shapes = list(shapes.shape_id.unique())
#     print(all_shapes)
    hqta = gpd.GeoDataFrame()
    for shape_id in bbb_test_shape_ids:
        print(shape_id)
#         display(hqta.tail(2))
        print('...')
        _result = single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop, shape_id, hqta)
        hqta = hqta.append(_result)
#         hqta = hqta.append(
#             single_shape_hqta(shapes, trips, stop_times, stops, route_count_by_stop, shape_id, hqta))
        hqta = hqta.set_crs('EPSG:6414')
        
    return hqta

In [337]:
# bbb_test_shape_ids = ['25387', '25386', '25311', '25313', '25314', '25315', '25316',
#        '25317', '25318', '25319', '25320', '25321', '25322', '25323',
#        '25324', '25325', '25326', '25330',
#        '25331', '25332', '25333', '25334', '25335', '25336', '25337',
#        '25339', '25340', '25341', '25342', '25343', '25344', '25346',
#        '25348', '25349', '25351', '25352', '25353', '25354', '25355',
#        '25356', '25357', '25358', '25359', '25361', '25362', '25363',
#        '25364', '25365', '25366', '25367', '25368', '25369', '25370',
#        '25371', '25372', '25374', '25375', '25376', '25379', '25380',
#        '25381', '25382', '25384', '25385', '25398', '25399', '25400',
#        '25402', '25403'] + ['25327', '25328', '25329']

In [338]:
bbb_test_shape_ids = bbb_views[0]['shape_id'].unique()
# bbb_test_shape_ids = ['25386', '25387', '25311', '25313', '25314', '25315', '25316']
# bbb_test_shape_ids = ['25311', '25313']
# bbb_test_shape_ids = ['25368']
# bbb_test_shape_ids = ['25386', '25387']


In [339]:
## passed for single op + single shape!
# bigtest = single_operator_hqta(300)

In [340]:
biggertest = single_operator_hqta(bbb_views)

25387
...
25387


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (7, 10)
25386
...
25386
no stops
25311
...
25311


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (10, 10)
25313
...
25313


...

appending (8, 10)
25382
...
25382


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (1, 10)
25384
...
25384


/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


appending (8, 10)
25385
...
25385
no line
25398
...
25398
no stops
25399
...
25399
no stops
25400
...
25400
no stops
25402
...
25402
no stops
25403
...
25403
no line


#### Flow

* segment table should be global for all operators...
* if shape contains existing segments, add them
* calculate new segments on non-overlap (which may be discontiguous?)
* should yield an operator+segment table that can be appended+grouped+summed with other operators...

### Mapping

In [341]:
biggertest.shape_id.unique()

array(['25387', '25311', '25313', '25314', '25315', '25318', '25321',
       '25322', '25327', '25330', '25331', '25336', '25342', '25346',
       '25353', '25354', '25362', '25365', '25367', '25381', '25382',
       '25384'], dtype=object)

In [342]:
hqta = biggertest

In [343]:
m = Map(basemap=basemaps.CartoDB.Positron, center=[34, -118.34], zoom=11)


In [344]:
geo_data_hq = GeoData(geo_dataframe = hqta[hqta['hq_transit_corr']].to_crs('EPSG:4326'),
                       style={'color': 'black', 'fillColor': '#3366cc',
                                    'opacity':0.3, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'HQTA')

geo_data_not_hq = GeoData(geo_dataframe = hqta[~hqta['hq_transit_corr']].to_crs('EPSG:4326'),
                       style={'color': 'black', 'fillColor': '#fec44f',
                                    'opacity':0.3, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                       hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                       name = 'non-HQTA')

In [345]:
m.add_layer(geo_data_hq)
m.add_layer(geo_data_not_hq)
m.add_control(LayersControl())

In [346]:
m

Map(center=[34, -118.34], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [349]:
_debug2

,calitp_itp_id,shape_id,calitp_extracted_at,geometry
21612,300,25402,2021-09-01,"MULTILINESTRING ((153531.830 -439465.107, 1535..."


In [352]:
_debug3['shape_id']

0      25384
1      25384
2      25384
3      25384
4      25384
       ...  
666    25384
667    25384
668    25384
669    25384
670    25384
Name: shape_id, Length: 671, dtype: object

In [300]:
def simple_map(gdf, mouseover=None):
    if 'calitp_extracted_at' in gdf.columns:
        gdf = gdf.drop(columns='calitp_extracted_at')
        gdf.geometry = gdf.geometry.buffer(50)
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[34, -118.34], zoom=11)

    if mouseover:
        html = HTML(f'hover to see {mouseover}')
        html.layout.margin = '0px 20px 20px 20px'
        control = WidgetControl(widget=html, position='topright')
        m.add_control(control)

        def update_html(feature,  **kwargs):
            html.value = '''
                <h3><b>{}</b></h3>
            '''.format(feature['properties'][mouseover])
    
    geo_data_hq = GeoData(geo_dataframe = gdf.to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#3366cc',
                                        'opacity':0.3, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.3},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                           name = 'gdf')
    m.add_layer(geo_data_hq)
    
    if mouseover:
        geo_data_hq.on_hover(update_html)

    return m

In [221]:
mapme = _debug[['shape_id', 'geometry']]
mapme.geometry = mapme.geometry.buffer(50)
simple_map(mapme)

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


Map(center=[34, -118.34], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [262]:
biggertest >> filter(_.shape_id.isin(['25327', '25328', '25329']))

,calitp_itp_id,shape_id,geometry,segment_sequence,hqta_segment_id,stop_id,n_routes,am_peak,pm_peak,hq_transit_corr
0,300,25327,"POLYGON ((138886.983 -443414.611, 138933.032 -...",0,2412214767,104,4,4.3,7.8,True
1,300,25327,"POLYGON ((139427.623 -443093.174, 139539.558 -...",1,4173351289,1217,7,20.0,25.5,True
2,300,25327,"POLYGON ((140018.067 -442721.119, 140018.321 -...",1,4173351289,255,4,20.0,25.5,True
3,300,25327,"POLYGON ((140449.619 -442290.515, 140449.744 -...",2,1640569027,1580,7,49.3,62.0,True
4,300,25327,"POLYGON ((141139.586 -441596.138, 141143.114 -...",2,1640569027,707,4,49.3,62.0,True
5,300,25327,"POLYGON ((141864.007 -441605.606, 141868.455 -...",2,1640569027,705,2,49.3,62.0,True
6,300,25327,"POLYGON ((142830.336 -441350.979, 142857.497 -...",2,1640569027,803,3,49.3,62.0,True
7,300,25327,"POLYGON ((143725.472 -440839.189, 143726.568 -...",2,1640569027,800,2,49.3,62.0,True
8,300,25327,"POLYGON ((144518.298 -440243.497, 144598.380 -...",2,1640569027,689,3,49.3,62.0,True
9,300,25327,"POLYGON ((145334.164 -439643.442, 145334.302 -...",2,1640569027,796,2,49.3,62.0,True


In [265]:
ids = biggertest >> filter(_.shape_id.isin(['25327', '25328', '25329']))
ids = ids['stop_id']

In [251]:
shapes, trips, stop_times, stops = get_operator_views(300)

In [256]:
routes = tbl.gtfs_schedule.routes() >> filter(_.calitp_itp_id == int(300)) >> collect()

In [252]:
shapes >> filter(_.)

,calitp_itp_id,calitp_url_number,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,calitp_extracted_at,geometry
0,300,0,25387,34.013189,-118.465589,37,2.6867,2021-09-01,POINT (141726.368 -443557.111)
1,300,0,25386,34.01593,-118.476129,89,5.0062,2021-09-01,POINT (140748.035 -443268.809)
2,300,0,25387,34.01322,-118.46768,34,2.4267,2021-09-01,POINT (141533.193 -443556.797)
3,300,0,25387,34.02348,-118.448259,70,4.6511,2021-09-01,POINT (143308.234 -442389.686)
4,300,0,25386,34.01683,-118.44451,16,0.5561,2021-09-01,POINT (143666.537 -443121.564)
...,...,...,...,...,...,...,...,...,...
22213,300,0,25403,34.048379,-118.542449,39,1.071,2021-09-01,POINT (134567.734 -439766.099)
22214,300,0,25403,34.04798,-118.526739,114,2.6709,2021-09-01,POINT (136018.725 -439787.936)
22215,300,0,25403,34.00963,-118.490299,332,10.8825,2021-09-01,POINT (139450.501 -443988.480)
22216,300,0,25403,34.04987,-118.53046,92,2.2391,2021-09-01,POINT (135671.962 -439583.638)


In [261]:
trips >> filter(_.route_id == '3332')

,calitp_itp_id,calitp_url_number,route_id,service_id,trip_id,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at
2046,300,0,3332,10,856796,25327,Palms Station,None,0,None,0,0,2021-09-01
2047,300,0,3332,10,856795,25327,Palms Station,None,0,None,0,0,2021-09-01
2048,300,0,3332,10,856798,25327,Palms Station,None,0,None,0,0,2021-09-01
2049,300,0,3332,10,856793,25327,Palms Station,None,0,None,0,0,2021-09-01
2050,300,0,3332,10,856797,25327,Palms Station,None,0,None,0,0,2021-09-01
2051,300,0,3332,10,856799,25327,Palms Station,None,0,None,0,0,2021-09-01
2052,300,0,3332,10,856792,25327,Palms Station,None,0,None,0,0,2021-09-01
2053,300,0,3332,10,856790,25327,Palms Station,None,0,None,0,0,2021-09-01
2054,300,0,3332,10,856800,25327,Palms Station,None,0,None,0,0,2021-09-01
2055,300,0,3332,10,856791,25327,Palms Station,None,0,None,0,0,2021-09-01


In [259]:
routes >> filter(_.route_short_name == '5')

,calitp_itp_id,calitp_url_number,route_id,route_type,agency_id,route_short_name,route_long_name,route_desc,route_url,route_color,route_text_color,route_sort_order,continuous_pickup,continuous_drop_off,calitp_extracted_at
3,300,0,3332,3,6216179,5,Olympic Blvd,None,http://bigbluebus.com/Routes-and-Schedules/Rou...,66B360,000000,None,None,None,2021-09-01


In [270]:
def map_stops(stops):
    gdf = gpd.GeoDataFrame(stops,
                 geometry = gpd.points_from_xy(stops.stop_lon, stops.stop_lat),
                 crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    gdf.geometry = gdf.geometry.buffer(25)
    return simple_map(gdf, 'stop_id')

In [275]:
map_stops(stops >> filter(_.stop_id.isin(ids)))

Map(center=[34, -118.34], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

investigate BBB 1082

In [318]:
stop_1082 = stop_times >> filter(_.stop_id == '1082')

In [319]:
stop_1082

,calitp_itp_id,calitp_url_number,trip_id,stop_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint,calitp_extracted_at
2621,300,0,856809,1082,4,15:17:39,15:17:39,None,0,0,None,None,1.4954,0,2021-09-01
2961,300,0,856812,1082,4,18:30:39,18:30:39,None,0,0,None,None,1.4954,0,2021-09-01
2965,300,0,856811,1082,4,17:26:39,17:26:39,None,0,0,None,None,1.4954,0,2021-09-01
3046,300,0,856806,1082,4,12:14:22,12:14:22,None,0,0,None,None,1.4954,0,2021-09-01
31330,300,0,856801,1082,3,07:15:21,07:15:21,None,0,0,None,None,1.2765,0,2021-09-01
32032,300,0,856802,1082,4,08:14:54,08:14:54,None,0,0,None,None,1.4954,0,2021-09-01
32227,300,0,856803,1082,4,09:14:39,09:14:39,None,0,0,None,None,1.4954,0,2021-09-01
60451,300,0,856804,1082,4,10:14:22,10:14:22,None,0,0,None,None,1.4954,0,2021-09-01
60803,300,0,856807,1082,4,13:14:22,13:14:22,None,0,0,None,None,1.4954,0,2021-09-01
60907,300,0,856808,1082,4,14:14:39,14:14:39,None,0,0,None,None,1.4954,0,2021-09-01


In [320]:
    ##only works for whole hours, but should be fine?
stop_1082['departure_hour'] = stop_1082['departure_time'].apply(lambda x:
                                                                dt.datetime.strptime(x, '%H:%M:%S').hour)

stop_1082 = stop_1082[stop_1082['departure_hour'].isin(commute_hours)]
stop_1082['am_peak'] = stop_1082['departure_hour'].apply(lambda x: x in am_commute_hours)
stop_1082['pm_peak'] = ~stop_1082['am_peak']

segment_peak_service = stop_1082.groupby(['stop_id'])[['am_peak', 'pm_peak']].sum()
segment_peak_service['am_peak'] = (segment_peak_service['am_peak'] / len(am_commute_hours)).round(1)
segment_peak_service['pm_peak'] = (segment_peak_service['pm_peak'] / len(pm_commute_hours)).round(1)

/Users/edasmalchi/anaconda3/envs/pd1/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [321]:
segment_peak_service

,am_peak,pm_peak
stop_id,,
1082,0.7,1.0


In [301]:
simple_map(_debug2)

Map(center=[34, -118.34], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…